In [49]:
import pandas as pd
import pickle
import gzip
import json

In [50]:
#Code adapted from the datasource http://jmcauley.ucsd.edu/data/amazon/
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [51]:
#Convert json to pd dataframe
df = getDF('Home_and_Kitchen.json.gz')

In [52]:
#Check datatype and information
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21928568 entries, 0 to 21928567
Data columns (total 12 columns):
 #   Column          Dtype  
---  ------          -----  
 0   overall         float64
 1   vote            object 
 2   verified        bool   
 3   reviewTime      object 
 4   reviewerID      object 
 5   asin            object 
 6   reviewerName    object 
 7   reviewText      object 
 8   summary         object 
 9   unixReviewTime  int64  
 10  style           object 
 11  image           object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 2.0+ GB


In [53]:
#View top 5 rows
df.head(5)

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image
0,5.0,2,True,"08 31, 2010",A3NSN9WOX8470M,0006564224,mmm,"I don't use these for their original use, and ...",Many uses...,1283212800,NaN,NaN
1,5.0,2,True,"04 2, 2010",A2AMX0AJ2BUDNV,0006564224,John R. Welch,"Seems a bit expensive for a plastic bottle, bu...",Dispenser bottle,1270166400,NaN,NaN
2,5.0,NaN,True,"11 5, 2015",A8LUWTIPU9CZB,0560467893,Linda Fahner,"Great product, love it!!",Five Stars,1446681600,NaN,NaN
3,4.0,4,True,"10 29, 2015",AABKIIHAL0L66,0560467893,TheBlueChain,This is a sturdy floating corner shelf! We mo...,"Sturdy Shelf, Poor Installation Instructions",1446076800,NaN,NaN
4,3.0,NaN,True,"09 9, 2015",A3DA0KIQ5OBK5C,0560467893,angelaarden,I purchased 4 of these shelves. they look grea...,Look great - one bad one...,1441756800,NaN,NaN


## Data Preparation and Cleaning
#### 1. The column name 'vote','style','image' does not provide much information for this project and 'reviewerName' is not required since the dataset contains 'reviewer id'
#### 2. Drop all the unverified reviews
#### 3. Conver reviewtime to date format 
#### 4. Find all duplicate entries (I.e If the reviewer has reviewed the same product more than once, Keep the latest review)
#### 5. Check for null values remove entries where both summary and reviewtext are nan
#### 6. Find all the url in summary text and replace them with ''
#### 7. Combine Review text and summary column and drop summary column and reset index
#### 8. Find all the url in review text and remove
#### 9. Check for null values
#### 10. Save pickle file

In [54]:
#Drop columns that are not required for analysis
df.drop(columns=['vote','style','image','reviewerName'],inplace=True)

In [55]:
#View top 5 rows
df.head(5)

,overall,verified,reviewTime,reviewerID,asin,reviewText,summary,unixReviewTime
0,5.0,True,"08 31, 2010",A3NSN9WOX8470M,0006564224,"I don't use these for their original use, and ...",Many uses...,1283212800
1,5.0,True,"04 2, 2010",A2AMX0AJ2BUDNV,0006564224,"Seems a bit expensive for a plastic bottle, bu...",Dispenser bottle,1270166400
2,5.0,True,"11 5, 2015",A8LUWTIPU9CZB,0560467893,"Great product, love it!!",Five Stars,1446681600
3,4.0,True,"10 29, 2015",AABKIIHAL0L66,0560467893,This is a sturdy floating corner shelf! We mo...,"Sturdy Shelf, Poor Installation Instructions",1446076800
4,3.0,True,"09 9, 2015",A3DA0KIQ5OBK5C,0560467893,I purchased 4 of these shelves. they look grea...,Look great - one bad one...,1441756800


In [56]:
#Check for the unique values in verified column
df.verified.unique()

array([ True, False])

In [57]:
#Drop all the unverified reviews
drp_ind = df[df['verified']==False].index
df.drop(drp_ind,inplace=True)

In [58]:
#Covert unixReview time to date date format and add a column 'rev_date'
df['rev_date'] = pd.to_datetime(df.unixReviewTime,unit = 's')

In [59]:
#drop unwanted columns
df.drop(columns=['reviewTime','unixReviewTime','verified'],inplace=True)

In [60]:
df

,overall,reviewerID,asin,reviewText,summary,rev_date
0,5.0,A3NSN9WOX8470M,0006564224,"I don't use these for their original use, and ...",Many uses...,2010-08-31
1,5.0,A2AMX0AJ2BUDNV,0006564224,"Seems a bit expensive for a plastic bottle, bu...",Dispenser bottle,2010-04-02
2,5.0,A8LUWTIPU9CZB,0560467893,"Great product, love it!!",Five Stars,2015-11-05
3,4.0,AABKIIHAL0L66,0560467893,This is a sturdy floating corner shelf! We mo...,"Sturdy Shelf, Poor Installation Instructions",2015-10-29
4,3.0,A3DA0KIQ5OBK5C,0560467893,I purchased 4 of these shelves. they look grea...,Look great - one bad one...,2015-09-09
...,...,...,...,...,...,...
21928563,3.0,A33Z5MOHLFIECI,B01HJH7K3Q,Not the size I was hoping for. Still a lovely ...,Three Stars,2017-12-21
21928564,5.0,A1BP93Y9L2HUL7,B01HJH7K3Q,Wonderful colors. Super smooth - great feel. ...,Wonderful colors. Super smooth - great feel,2017-04-26
21928565,5.0,A3EYW1FBJ48SH1,B01HJH7K3Q,"Excellent Quality, shipping did not take near ...",Excellent Quality,2017-01-03
21928566,5.0,AYOGJY5CDIY49,B01HJHTC6O,I bumped my old one and shattered it. This was...,I bumped my old one and shattered it. This ...,2018-08-03


In [14]:
#Sorting the rows by rev_date , old reviews to latest reviews
df = df.sort_values(by = 'rev_date')

In [15]:
#ADD CODE for finding duplicates
df[df.duplicated(subset = ['asin','reviewerID'], keep = 'last')]

,overall,reviewerID,asin,reviewText,summary,rev_date
49831,5.0,ALJGB7XHPFCR,B00004R92U,After mixing your dough and letting it rise th...,BETTER THAN SLICED BREAD!,2000-06-30
50187,5.0,ALJGB7XHPFCR,B00004R92U,After mixing your dough and letting it rise th...,A great bread-making tool,2000-07-11
106985,5.0,A231WM2Z2JL0U3,B00004SQ0K,"I've had mine for quite some time, and I would...",Perfect Results Always.,2000-09-05
66223,5.0,AX54G5AL870Q8,B00004RG94,Why do I like these so much? Because the othe...,My favorite set of measuring spoons.,2000-10-19
100593,4.0,AJF1X7RTIDD8Y,B00004SDTC,This toaster toasts just fine. The bread come...,"It's a fine toaster, but",2001-06-22
...,...,...,...,...,...,...
21620553,5.0,AVWHLU1C2OWMD,B01D1ZUF78,"Keeps hot enough to be drinkable, unlike the M...","Keeps hot enough to be drinkable, unlike the M...",2018-09-07
21620552,5.0,AVWHLU1C2OWMD,B01D1ZUF78,keeps my tea and candles warm!,Five Stars,2018-09-07
21620551,5.0,AVWHLU1C2OWMD,B01D1ZUF78,Five StarsWorks just as described. Happy I can...,Happy I can finally drink my tea without it ge...,2018-09-07
17130677,3.0,AV88XFKP7MXNS,B01HE1IVNA,Ok product,Ok product,2018-09-08


In [16]:
#Deleting these duplicate values keeping the latest review (the last value found)
df.drop_duplicates(subset = ['asin','reviewerID'], keep ='last', inplace = True)

In [17]:
df

,overall,reviewerID,asin,reviewText,summary,rev_date
105246,5.0,A84J7PUWOC138,B00004SPEM,"For drip coffee at home, you can't go wrong wi...",Good home coffee maker,2000-05-04
48581,4.0,A2MNB77YGJ3CN0,B00004R940,I'll admit it -- I'm a bit of a snob when it c...,"I was skeptical, but now I'm sold . . .",2000-05-07
64286,5.0,A2Y8K7GP1QXUF4,B00004RFLJ,This is perfection. You will never use a bett...,perfect,2000-05-15
97918,5.0,A3MDOV15JHNLB2,B00004SAF7,This corkscrew is amazing. There are several ...,simply the best,2000-05-16
83109,5.0,A1LUUO72VAFKFJ,B00004S4TZ,Maybe when Mom cooked for a big family every d...,Superb. Why didn't somebody think of this yea...,2000-05-19
...,...,...,...,...,...,...
21922967,5.0,ATLLCVVXJLM20,B01HGSBY1M,I love my robot and want to make sure he alway...,Wonderful filters,2018-10-04
21924579,5.0,A9DVSUU814KHG,B01HHDNCJS,Beautiful knife and packaging. Made a great gift,Well made,2018-10-04
21924615,2.0,A1NUE7VF4F83XK,B01HHDN586,The base of the bottle cap leaks bad especiall...,Bottle is leaking when squeezed,2018-10-04
21927348,5.0,A1DNLCUGJSDS8R,B01HISXWU6,Beautiful stones. Larger then I expected. Ve...,Beautiful,2018-10-04


In [18]:
df.isna().sum()

overall           0
reviewerID        0
asin              0
reviewText    13782
summary        5360
rev_date          0
dtype: int64

In [19]:
#Check if both summary and review text are nan, delete those rows
cols = ['reviewText','summary']
df[df[cols].isnull().all(axis=1)]

,overall,reviewerID,asin,reviewText,summary,rev_date
8574259,3.0,A31XB44TOW2RYC,B00BU2RPZK,NaN,NaN,2014-07-10
7215727,5.0,A31XB44TOW2RYC,B007HQ0MGW,NaN,NaN,2014-07-10
9944607,5.0,A3DHCIDXXT0GO8,B00GCAHDJ8,NaN,NaN,2014-08-31
2309307,4.0,A2XX64MZUGGR35,B000XU49MW,NaN,NaN,2014-09-04
4115595,5.0,A3DHCIDXXT0GO8,B00240AIIS,NaN,NaN,2014-09-08
...,...,...,...,...,...,...
21784560,5.0,A3RD507HMSLQJ2,B01FH8O37K,NaN,NaN,2018-09-24
21814305,5.0,A2Y6CNDAYR38ED,B01FX5CMC0,NaN,NaN,2018-09-26
21829935,5.0,AO0VEUDFJTNY,B01G58S2HS,NaN,NaN,2018-09-27
21903959,4.0,AUXX9QGK2ROAM,B01H6WX47A,NaN,NaN,2018-09-27


In [20]:
idx = df[df[cols].isnull().all(axis=1)].index

In [21]:
df.drop(idx,inplace=True)

In [22]:
#Get all the indexes of reviewtext which has url
df.summary[df.summary.str.match(pat = '(https)|(http)|(www.)',na = False)]

3192758     http://www.amazon.com/gp/product/B00185XISW/re...
14545751    http://www.amazon.com/gp/product/B0006HUYGM/re...
3274283     http://www.amazon.com/gp/product/B0019S20CS/re...
3111557     http://www.amazon.com/gp/product/B0016NQ49Q/re...
256928      http://www.amazon.com/gp/product/B0000631ZM/re...
                                  ...                        
19407716    https: //www. amazon. com/dp/B00FRGFSH2/ref=cm...
19707710                                https: //www. amazon.
21901732                                https: //www. amazon.
19152212                                https: //www. amazon.
21573556                                https: //www. amazon.
Name: summary, Length: 779, dtype: object

In [23]:
#store the index
ids = df.summary[df.summary.str.match(pat = '(https)|(http)|(www.)',na = False)].index

In [24]:
#display a sample summary with url
df.summary[3274283]

'http://www.amazon.com/gp/product/B0019S20CS/ref=cm_cr_rev_prod_title'

In [25]:
#Replace url with ''
for i in ids:
    df.summary[i]=''

/home/014544907/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [26]:
#double check all summary if it still has url
id_s = df[df.summary.str.match(pat = '(https)|(http)|(www.)',na = False)].index

In [27]:
#Index of all the summary which has urls
id_s

Int64Index([], dtype='int64')

In [28]:
#combine both review and summary texts
df['reviewText'] = df[['reviewText', 'summary']].astype(str).apply(lambda x: ' //'.join(x), axis = 1)

In [29]:
#deleting summary column
del df['summary']

In [30]:
#reset index
df.reset_index(inplace=True,drop=True)

In [31]:
df

,overall,reviewerID,asin,reviewText,rev_date
0,5.0,A84J7PUWOC138,B00004SPEM,"For drip coffee at home, you can't go wrong wi...",2000-05-04
1,4.0,A2MNB77YGJ3CN0,B00004R940,I'll admit it -- I'm a bit of a snob when it c...,2000-05-07
2,5.0,A2Y8K7GP1QXUF4,B00004RFLJ,This is perfection. You will never use a bett...,2000-05-15
3,5.0,A3MDOV15JHNLB2,B00004SAF7,This corkscrew is amazing. There are several ...,2000-05-16
4,5.0,A1LUUO72VAFKFJ,B00004S4TZ,Maybe when Mom cooked for a big family every d...,2000-05-19
...,...,...,...,...,...
19525622,5.0,ATLLCVVXJLM20,B01HGSBY1M,I love my robot and want to make sure he alway...,2018-10-04
19525623,5.0,A9DVSUU814KHG,B01HHDNCJS,Beautiful knife and packaging. Made a great gi...,2018-10-04
19525624,2.0,A1NUE7VF4F83XK,B01HHDN586,The base of the bottle cap leaks bad especiall...,2018-10-04
19525625,5.0,A1DNLCUGJSDS8R,B01HISXWU6,Beautiful stones. Larger then I expected. Ve...,2018-10-04


In [32]:
#code for removing url in any form
import re
def remove_urls (vTEXT):
    vTEXT = re.sub(r'(https|http|www)(://|\.)(\w|\.|\/|\?|\=|\&|\%|[-]|\///)*\b', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)

In [33]:
#Get all the indexes of reviewtext which has url
idrev = df[df.reviewText.str.match(pat = '(https)|(http)|(www.)')].index

In [34]:
#Index of all the reviews which has urls
idrev

Int64Index([   74860,    75308,    75824,    84431,    85987,    87209,
               90895,    93883,    96131,    99861,
            ...
            19022204, 19024435, 19067368, 19129718, 19222268, 19223852,
            19224882, 19239353, 19355583, 19519064],
           dtype='int64', length=1022)

In [35]:
#Check a sample review test with url
df.reviewText[74860]

'http://www.amazon.com/gp/r.html?R=29X6S3WLIEYM7&C=3V8D31BXPAK8Q&H=RzFWJcgAnKz398zzIJrJCIuP33UA&T=C&U=http%3A%2F%2Fwww.amazon.com%2Freview%2Fcreate-review%2Fref%3Dpe_6680_10397670_cm_cr_ec_add_1_h_c10397670%3Fie%3DUTF8%26nodeID%3D%26asin%3DB000K6LLMA%26customerID%3DA3U3VIYMKE5CUX\nThis collar was extremely too big!!  It is faulty and did not work well on the animal.  It went off when it felt like it and it is very loud to the human ear!  My dog just barked louder so I could here her over the beeping noise.  I would not recommend this product to anyone with a small dog the box is way too big for a small dog. //dog collar'

In [36]:
#After removing url
remove_urls(df.reviewText[74860])

'\nThis collar was extremely too big!!  It is faulty and did not work well on the animal.  It went off when it felt like it and it is very loud to the human ear!  My dog just barked louder so I could here her over the beeping noise.  I would not recommend this product to anyone with a small dog the box is way too big for a small dog. //dog collar'

In [37]:
#Remove url for all the indexes which had url
for i in idrev:
    df.reviewText[i] = remove_urls(df.reviewText[i])

/home/014544907/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [38]:
#Double check if reviewtext still contains http/www
idrev = df[df.reviewText.str.match(pat = '(https)|(http)|(www.)')].index

In [39]:
#check index
idrev

Int64Index([5432942, 11807698], dtype='int64')

In [40]:
#These are not removed, because they are not actually urls
print(df.reviewText[11807784])
print(df.reviewText[5432958])

I love these liners!!! I cut one of sheets smaller to fit in my toaster oven and it works perfect! I'm in love! //I love these liners
Was extremely pleased with this knife sharpener, it sharpens kn //Five Stars


In [41]:
#Check for null values
df.isnull().sum()

overall       0
reviewerID    0
asin          0
reviewText    0
rev_date      0
dtype: int64

In [42]:
df

,overall,reviewerID,asin,reviewText,rev_date
0,5.0,A84J7PUWOC138,B00004SPEM,"For drip coffee at home, you can't go wrong wi...",2000-05-04
1,4.0,A2MNB77YGJ3CN0,B00004R940,I'll admit it -- I'm a bit of a snob when it c...,2000-05-07
2,5.0,A2Y8K7GP1QXUF4,B00004RFLJ,This is perfection. You will never use a bett...,2000-05-15
3,5.0,A3MDOV15JHNLB2,B00004SAF7,This corkscrew is amazing. There are several ...,2000-05-16
4,5.0,A1LUUO72VAFKFJ,B00004S4TZ,Maybe when Mom cooked for a big family every d...,2000-05-19
...,...,...,...,...,...
19525622,5.0,ATLLCVVXJLM20,B01HGSBY1M,I love my robot and want to make sure he alway...,2018-10-04
19525623,5.0,A9DVSUU814KHG,B01HHDNCJS,Beautiful knife and packaging. Made a great gi...,2018-10-04
19525624,2.0,A1NUE7VF4F83XK,B01HHDN586,The base of the bottle cap leaks bad especiall...,2018-10-04
19525625,5.0,A1DNLCUGJSDS8R,B01HISXWU6,Beautiful stones. Larger then I expected. Ve...,2018-10-04


In [43]:
#Check if overall contains valid rating
df.overall.unique()

array([5., 4., 2., 1., 3.])

In [48]:
df[df.reviewText.str.contains(' NaN ')==True]

,overall,reviewerID,asin,reviewText,rev_date


In [44]:
df.to_pickle('Clean_data')